In [98]:
# Import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
from types import *
import matplotlib.animation as animation
from matplotlib import rc
import pandas as pd

In [99]:
para_vitality = pd.read_excel('../fig/concentration_vitality.xlsx')
para_vitality.iloc[1,2]

-9.83e-05

Functions

In [100]:
# No changes are needed for this cell.
figSize=600

# Let's define a simple function that computes the time of the next reaction given our propensity functions:
def next_time(x, t, W_SSA, pars, di, ti):
    return -np.log(np.random.rand()) / np.sum(W_SSA(x, t, pars, di, ti))

# Let's define a simple function that computes the index of the next reaction given our propensity functions:
def next_reaction(x, t, W_SSA, pars, di, ti):
    W = W_SSA(x, t, pars, di, ti)
    W0 = np.sum(W)
    r = np.random.rand()
    i = 0
    W_sum = W[0]
    while W_sum / W0 < r:
        i += 1
        W_sum += W[i]
    return i

def gillespie_drugs(x0, t0, tmax, S, W_SSA, pars, di, ti, returnFullTraj = False):
    # Initialize the time and the state
    t = t0
    x = np.array(x0)
    S = np.array(S)
    
    # Initialize the output (if requested)
    if returnFullTraj:
        times = [t0]
        states = [x0]
    
    # Run the simulation
    while t < tmax:

        # Compute the time of the next reaction
        tau = next_time(x, t, W_SSA, pars, di, ti)
        t += tau
        
        # Exit the loop if the next reaction is beyond tmax
        if t > tmax:
            break
        
        # Compute the index of the next reaction
        i = next_reaction(x, t, W_SSA, pars, di, ti)
        # Update the state
        x = x + S[:, i]
        
        # Append the results (if requested)
        if returnFullTraj:
            times.append(t)
            states.append(x)

    # Return the results
    if returnFullTraj:
        # Return the full trajectory
        return np.array(times), np.array(states)
    else:
        # Return only the final state
        return x
    
def dxdt(x, t, pars, di, ti):
    x = np.atleast_2d(x).T
    y = (S @ W_ODE(x, t, pars, di, ti)).flatten()
    return y


def solve(S, W_ODE, W_SSA, x0, t, pars, di, ti, model='ODE', ntraj=1):

    if model == 'ODE':
        trajectory = odeint(dxdt,np.array(x0),t, args=(pars, di, ti))

    if model == 'SSA':
        trajectory = np.zeros([ntraj, len(t), len(x0)])
        t0 = t[0]
        tmax = t[-1]
        for i in range(ntraj):
            times,states = gillespie_drugs(x0, t0, tmax, S, W_SSA, pars, di, ti, returnFullTraj=True)
            
            k = 0
            for j in range(len(times)):
                while times[j] > t[k]:
                    trajectory[i,k,:] = states[j]
                    k+=1

        trajectory = np.mean(trajectory, axis=0)
    return trajectory


def compare_model_to_data(pars,di,ti, x0, model='ODE', ntraj=10):
    s = solve(S, W_ODE, W_SSA, x0, np.linspace(0,2000,2001),pars, di, ti, model=model, ntraj=ntraj )
    return s

# Propensity functions
def W_ODE(x,t,pars,di,ti):
    return np.array([np.array([0.1209])*(t<ti[2]) + pars['k1']*(t>=ti[2]),
                     pars['k2']*x[0],
                     (0.0162*(t<ti[0]) + pars['k3']*(t>=ti[0]))*x[1],
                     (0.9610*(t<ti[1]) + pars['k4']*(t>=ti[1]))*x[2],
                     (0.01*(t<ti[4]) + pars['k5']*(t>=ti[4]))*x[3],
                     (0.009*(t<ti[3]) + pars['k6']*(t>=ti[3]))*x[2],
                     (0.0109*(t<ti[3]) + pars['k7']*(t>=ti[3]))*x[1]])

def W_SSA(x,t,pars,di,ti):
    return np.array([0.1209*(t<ti[2]) + pars['k1']*(t>=ti[2]),
                     pars['k2']*x[0],
                     (0.0162*(t<ti[0]) + pars['k3']*(t>=ti[0]))*x[1],
                     (0.9610*(t<ti[1]) + pars['k4']*(t>=ti[1]))*x[2],
                     (0.01*(t<ti[4]) + pars['k5']*(t>=ti[4]))*x[3],
                     (0.009*(t<ti[3]) + pars['k6']*(t>=ti[3]))*x[2],
                     (0.0109*(t<ti[3]) + pars['k7']*(t>=ti[3]))*x[1]])

def J(s,trajectory_f,di,ti):
    J = (trajectory_f[-1]/trajectory_f[1000] + (s.T[speciesDict['Protein']][1000]-s.T[speciesDict['Protein']][-1])/s.T[speciesDict['Protein']][1000]) / (1 + np.sum(di)/1000)
    return J

def W_f(x,t,di,ti,ri):
    W_f = (0.000072+ri[0]*(t>=ti[0])+ri[1]*(t>=ti[1])+ri[2]*(t>=ti[2])+ri[3]*(t>=ti[3])+ri[4]*(t>=ti[4]))*x[0]
    return np.array([W_f])

def dxdt_f(x,t,di,ti,ri):
    y = (S_f @ W_f(x, t, di, ti, ri)).flatten()
    return y

Optimization of J

In [101]:

di = [0,0,0,0,0]  # concentrations of the drugs
# [Drug2, Drug4, Drug6, Drug7, Drug10]
ti = [np.inf,np.inf,np.inf,np.inf,np.inf] # times at which the drugs are added
# [Drug2, Drug4, Drug6, Drug7, Drug10]
x0_f = np.array([1]) # initial conditions
ri = [] # vitality decrease rate of each drug
# [Drug2, Drug4, Drug6, Drug7, Drug10]

for i in range(5):
    ri.append(para_vitality.iloc[0,i+1]*di[i] + para_vitality.iloc[1,i+1])

# Define Function f(t)
S_f = np.array([-1])

trajectory_f = odeint(dxdt_f, x0_f, np.linspace(0,2000,2001) , args=(di, ti, ri))

x0 = np.array([6.8, 4.6, 8.5, 880]) # initial conditions

# Parameters
# pars = {'k1': 0.003, 'k2': 0.019, 'k3': 0.0162, 'k4': 0.9610, 'k5': 0.01, 'k6': 0.009, 'k7': 0.0109} # NoDrug
pars = {'k1': -0.121*di[2]**2.363/(299.132**2.363+di[2]**2.363)+0.12, 
        'k2': 0.019, 
        'k3': -0.016*di[0]**3.085/(480.176**3.085+di[0]**3.085)+0.016, 
        'k4': -0.968*di[1]**3.112/(695.525**3.112+di[1]**3.112)+0.969,
        'k5': 1.208*di[4]**1.781/(5338.262**1.781+di[4]**1.781)+0.008, 
        'k6': 0.306*di[3]**2.061/(2031.863**2.061+di[3]**2.061)+0.009, 
        'k7': 0.306*di[3]**2.061/(2031.863**2.061+di[3]**2.061)+0.009}

# Species dictionary - for plotting
speciesDict = {'Nascent':0, 'Nuclear':1, 'Cytoplasmic':2, 'Protein':3}  # how to map your species to the data 

# The stoichiometry matrix
S = np.array([[1,-1,0,0,0,0,0],
              [0,1,-1,0,0,0,-1],
              [0,0,1,0,0,-1,0],
              [0,0,0,1,-1,0,0]])

s = compare_model_to_data(pars,di,ti, x0, model='ODE', ntraj=10)
r = J(s,trajectory_f,di,ti)